# Metadata

**L1 Taxonomy** - SQL in Python

**L2 Taxonomy** - CRUD Operations

**Subtopic** - Implementing CRUD Operations in Python Applications using SQLAlchemy and SQLite

**Use Case** - Develop a stand-alone Python application that uses SQLAlchemy and SQLite to implement CRUD (Create, Read, Update, Delete) operations. The user shall be able to add, view, modify, and delete records in a SQLite database. Input and output operations will be handled using standard terminal inputs and outputs.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
sqlalchemy==2.0.30
```


# Prompt
You are building a standalone command-line application that allows users to manage a digital library using basic CRUD (Create, Read, Update, Delete) operations. The system must use SQLAlchemy ORM for all database interactions and SQLite as the backend. The application should store books in a database and allow users to interact via the terminal.

---

Functional Requirements

-	Users can enter commands: add, view, update, delete, and exit.
-	add should collect a book’s title, author, and year of publication from the user.
-	view should print all books in a structured tabular format with headers.
-	update should prompt for a valid book ID and allow updating any or all of its fields.
-	delete should remove a book by its ID.
-	If a book ID does not exist or if any field is invalid, the program should not crash and should print an appropriate message.

The application must use standard input/output functions like input() and print(). It should be structured so that output can be captured and tested easily. The LibraryManager class must internally initialize the database and session. The constructor must not take any parameters. During testing, sessions can be replaced manually for isolation.

----

Schema
```
class Book(Base):
    __tablename__ = 'books'
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    author = Column(String, nullable=False)
    year_published = Column(Integer, nullable=False)

```

```
class LibraryManager:
    def __init__(self): ...
    def add_book(self, title, author, year): ...
    def view_books(self): ...
    def update_book(self, book_id, title=None, author=None, year=None): ...
    def delete_book(self, book_id): ...
    def close_session(self): ...
```

# Requirements

Requirements
-	Use SQLAlchemy’s ORM (not raw SQL) for defining the data model and performing all database operations.
-	Use a file-based SQLite database stored as library.db.
-	Define a Book model with the fields: id (auto-incremented primary key), title (string), author (string), and year_published (integer ≥ 0).
-	The main interface logic must be implemented through a LibraryManager class.
-	LibraryManager should include the following methods:
  -	add_book(title, author, year)
	-	view_books()
	-	update_book(book_id, title=None, author=None, year=None)
	-	delete_book(book_id)
	-	close_session()
-	The constructor of LibraryManager must take no parameters and should initialize the SQLite engine internally with library.db as the default.
-	Output formatting, such as the book list table, should use clear section headers like “Library Collection:” or similar.
-	The CLI must gracefully handle edge cases such as empty or whitespace-only input, non-existent book IDs, re-adding a deleted book, and attempting to delete a book twice.
-	It is expected that each book operation is confirmed with a printed message, such as:
	-	Book added: Title by Author (Year)
	-	Book updated: ID <id>
	-	Book with ID <id> deleted
	-	or No book found with ID <id> when applicable.

The application should be written in a way that allows isolated testing by redirecting standard output and directly manipulating the database session when needed. Each method should produce console output that can be captured and asserted during unit tests without requiring user interaction.

In [ ]:
# code

from typing import List, Optional, Union
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.engine import Engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.orm import Session, declarative_base, sessionmaker

Base = declarative_base()

class Book(Base):
    """
    ORM model for the 'books' table.
    Each row represents a single book with title, author, and year of publication.
    """
    __tablename__ = "books"
    __table_args__ = {"sqlite_autoincrement": True}

    id: int = Column(Integer, primary_key=True, autoincrement=True)
    title: str = Column(String, nullable=False)
    author: str = Column(String, nullable=False)
    year_published: int = Column(Integer, nullable=False)

class LibraryManager:
    """
    Handles all operations related to managing the digital library.
    Supports adding, viewing, updating, and deleting books using SQLAlchemy ORM.
    """

    def __init__(self, db_ref: Union[str, Engine] = "sqlite:///library.db"):
        """
        Initializes the LibraryManager with a SQLite database.
        Automatically creates the 'books' table if it doesn't exist.
        """
        try:
            self.engine: Engine = (
                create_engine(db_ref, echo=False) if isinstance(db_ref, str) else db_ref
            )
            Base.metadata.create_all(self.engine)
            self.Session = sessionmaker(bind=self.engine)
            self.session: Session = self.Session()
        except SQLAlchemyError as exc:
            raise RuntimeError(f"Database setup failed: {exc}") from exc

    def _int_or_error(self, value: Union[int, str]) -> int:
        """
        Tries to convert the given value into an integer.
        Raises ValueError if the conversion fails.
        """
        try:
            return int(value)
        except (ValueError, TypeError):
            raise ValueError("Year must be an integer.")

    def add_book(self, title: str, author: str, year: Union[int, str]) -> None:
        """
        Adds a new book to the database after validating inputs.
        Rejects empty title/author and negative or invalid years.
        """
        if not title.strip():
            print("Title cannot be empty")
            return
        if not author.strip():
            print("Author cannot be empty")
            return

        try:
            year_int = self._int_or_error(year)
        except ValueError:
            print("Invalid year")
            return

        if year_int < 0:
            print("Invalid year")
            return

        book = Book(title=title.strip(), author=author.strip(), year_published=year_int)
        self.session.add(book)
        self.session.commit()
        print(f"Book added: {book.title} by {book.author} ({book.year_published})")

    def view_books(self) -> None:
        """
        Displays all books currently stored in the library in a formatted table.
        Sorted by ID in ascending order.
        """
        books: List[Book] = self.session.query(Book).order_by(Book.id).all()

        print("\nLibrary Collection:")
        print(f" {'ID':<3} | {'Title':<20} | {'Author':<20} | Year")
        print(f"-{'-'*3}-+-{'-'*20}-+-{'-'*20}-+------")

        for b in books:
            print(f" {b.id:<3} | {b.title:<20} | {b.author:<20} | {b.year_published}")
        print()

    def update_book(
        self,
        book_id: int,
        title: Optional[str] = None,
        author: Optional[str] = None,
        year: Optional[Union[int, str]] = None,
    ) -> None:
        """
        Updates the title, author, or year of a specific book.
        Only the fields provided will be updated. Skips others.
        Prints a message if the book doesn't exist or inputs are invalid.
        """
        book = self.session.query(Book).filter_by(id=book_id).first()
        if not book:
            print(f"No book found with ID {book_id}")
            return

        if title is not None:
            if not title.strip():
                print("Title cannot be empty")
                return
            book.title = title.strip()

        if author is not None:
            if not author.strip():
                print("Author cannot be empty")
                return
            book.author = author.strip()

        if year is not None:
            try:
                year_int = self._int_or_error(year)
            except ValueError:
                print("Invalid year")
                return

            if year_int < 0:
                print("Invalid year")
                return
            book.year_published = year_int

        self.session.commit()
        print(f"Book updated: {book_id}")

    def delete_book(self, book_id: int) -> None:
        """
        Deletes a book by its ID from the database.
        If the ID does not exist, a message is shown.
        """
        book = self.session.query(Book).filter_by(id=book_id).first()
        if not book:
            print(f"No book found with ID {book_id}")
            return
        self.session.delete(book)
        self.session.commit()
        print(f"Book with ID {book_id} deleted.")

    def close_session(self) -> None:
        """
        Safely closes the current database session.
        Should be called before exiting the application.
        """
        self.session.close()

In [ ]:
# tests

import unittest
from unittest.mock import patch
import io, sys
from sqlalchemy import create_engine
from main import LibraryManager, Base

class TestLibraryManagerCLI(unittest.TestCase):
    def setUp(self):
        self._stdout = sys.stdout
        self.buf = io.StringIO()
        sys.stdout = self.buf

        # Patch create_engine to use in-memory SQLite
        patcher = patch('main.create_engine')
        self.addCleanup(patcher.stop)
        mock_create = patcher.start()
        self.engine = create_engine('sqlite:///:memory:')
        mock_create.return_value = self.engine

        # Initialize in-memory DB
        Base.metadata.create_all(self.engine)

        # Instantiate manager (uses patched engine)
        self.lm = LibraryManager()

    def tearDown(self):
        sys.stdout = self._stdout

    def test_add_book_success(self):
        self.lm.add_book("1984", "George Orwell", 1949)
        out = self.buf.getvalue()
        self.assertIn("Book added: 1984 by George Orwell (1949)", out)

    def test_view_books_empty(self):
        self.lm.view_books()
        out = self.buf.getvalue()
        self.assertIn("Library Collection:", out)
        self.assertIn("ID", out)
        # No book rows
        rows = [line for line in out.splitlines() if line.startswith("|")]
        self.assertEqual(rows, [])

    def test_view_books_with_entries(self):
        self.lm.add_book("Dune", "Frank Herbert", 1965)
        self.buf.truncate(0); self.buf.seek(0)
        self.lm.view_books()
        out = self.buf.getvalue()
        self.assertIn("Dune", out)
        self.assertIn("Frank Herbert", out)
        self.assertIn("1965", out)

    def test_update_book_success(self):
        self.lm.add_book("Old Title", "Author", 2000)
        self.buf.truncate(0); self.buf.seek(0)
        self.lm.update_book(1, title="New Title")
        out = self.buf.getvalue()
        self.assertIn("Book updated: 1", out)

    def test_update_book_nonexistent(self):
        self.lm.update_book(99, author="X")
        out = self.buf.getvalue()
        self.assertIn("No book found with ID 99", out)

    def test_delete_book_success(self):
        self.lm.add_book("Temp", "Writer", 2021)
        self.buf.truncate(0); self.buf.seek(0)
        self.lm.delete_book(1)
        out = self.buf.getvalue()
        self.assertIn("Book with ID 1 deleted", out)

    def test_delete_book_twice(self):
        self.lm.add_book("Temp", "Writer", 2021)
        self.lm.delete_book(1)
        self.buf.truncate(0); self.buf.seek(0)
        self.lm.delete_book(1)
        out = self.buf.getvalue()
        self.assertIn("No book found with ID 1", out)

    def test_add_book_invalid_year(self):
        self.lm.add_book("Title", "Author", -5)
        out = self.buf.getvalue()
        self.assertIn("Invalid year", out)

    def test_add_book_empty_title(self):
        self.lm.add_book("", "Author", 2020)
        out = self.buf.getvalue()
        self.assertIn("Title cannot be empty", out)

    def test_update_book_invalid_year(self):
        self.lm.add_book("Valid", "Author", 2020)
        self.buf.truncate(0); self.buf.seek(0)
        self.lm.update_book(1, year=-1)
        out = self.buf.getvalue()
        self.assertIn("Invalid year", out)

    def test_close_session(self):
        try:
            self.lm.close_session()
        except Exception as e:
            self.fail(f"close_session raised {e}")

if __name__ == '__main__':
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/8419f37b-34c8-4c6e-99f9-9198397ac05c

#### Model code:

```python
# code generated by the model

import sys
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Book(Base):
    __tablename__ = 'books'
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    author = Column(String, nullable=False)
    year_published = Column(Integer, nullable=False)

class LibraryManager:
    def __init__(self):
        self.engine = create_engine('sqlite:///library.db')
        Base.metadata.create_all(self.engine)
        Session = sessionmaker(bind=self.engine)
        self.session = Session()  # <Issue>: tightly coupled session, not testable

    def add_book(self, title, author, year):  # <Issue>: missing type hints >>>
        try:
            year_int = int(year)
            if year_int < 0:
                print("Year must be a non-negative integer.")
                return
        except ValueError:
            print("Invalid year entered.")
            return

        if not title.strip() or not author.strip():
            print("Title and Author must not be empty.")
            return

        new_book = Book(title=title.strip(), author=author.strip(), year_published=year_int)
        self.session.add(new_book)
        self.session.commit()
        print(f"Book added: {title.strip()} by {author.strip()} ({year_int})")

    def view_books(self):  # <Issue> missing type hints
        books = self.session.query(Book).all()
        if not books:
            print("No books found.")
            return
        print("Library Collection:")
        print(f"{'ID':<5} {'Title':<30} {'Author':<30} {'Year':<5}")
        for b in books:
            print(f"{b.id:<5} {b.title:<30} {b.author:<30} {b.year_published:<5}")
        # <Issue>: no handling for overly long fields or very large book list (pagination/help header)

    def update_book(self, book_id, title=None, author=None, year=None):  # <Issue>: missing type hints
        try:
            book = self.session.query(Book).get(book_id)  # <Issue>: .get() is deprecated in SQLAlchemy 2.x
        except:
            print(f"Error retrieving book with ID {book_id}.")  # <Issue>: bare except used
            return

        if not book:
            print(f"No book found with ID {book_id}.")
            return

        updated = False
        if title is not None:
            if not title.strip():
                print("Title must not be empty. Ignoring title update.")
            else:
                book.title = title.strip()
                updated = True
        if author is not None:
            if not author.strip():
                print("Author must not be empty. Ignoring author update.")
            else:
                book.author = author.strip()
                updated = True
        if year is not None:
            try:
                year_int = int(year)
                if year_int < 0:
                    print("Year must be a non-negative integer. Ignoring year update.")
                else:
                    book.year_published = year_int
                    updated = True
            except ValueError:
                print("Invalid year entered. Ignoring year update.")

        if updated:
            self.session.commit()
            print(f"Book updated: {book_id}")
        else:
            print("No valid fields to update.")

    def delete_book(self, book_id):  # <Issue>: missing type hints
        try:
            book = self.session.query(Book).get(book_id)  # <<< ISSUE: .get() is deprecated
        except:
            print(f"Error deleting book with ID {book_id}.")  # <Issue>: bare except used
            return

        if not book:
            print(f"No book found with ID {book_id}.")
            return

        self.session.delete(book)
        self.session.commit()
        print(f"Book with ID {book_id} deleted.")

    def close_session(self):  # <Issue>: missing type hints
        self.session.close()

def main():
    manager = LibraryManager()
    while True:
        command = input("Enter command (add/view/update/delete/exit): ").strip().lower()
        if not command:
            print("Empty input. Please enter a command.")  # <Issue>: this behavior isn't present, but should be
            continue

        if command == 'add':
            title = input("Title: ")
            author = input("Author: ")
            year = input("Year of publication: ")
            manager.add_book(title, author, year)
        elif command == 'view':
            manager.view_books()
        elif command == 'update':
            book_id = input("Enter the book ID to update: ").strip()
            if not book_id.isdigit():
                print("Invalid book ID.")
                continue
            book_id = int(book_id)
            new_title = input("New title (leave blank to skip): ")
            new_author = input("New author (leave blank to skip): ")
            new_year = input("New year (leave blank to skip): ")
            manager.update_book(book_id,
                                title=new_title if new_title else None,
                                author=new_author if new_author else None,
                                year=new_year if new_year else None)
        elif command == 'delete':
            book_id = input("Enter the book ID to delete: ").strip()
            if not book_id.isdigit():
                print("Invalid book ID.")
                continue
            manager.delete_book(int(book_id))
        elif command == 'exit':
            manager.close_session()
            print("Exiting application.")
            sys.exit(0)
        else:
            print("Unknown command. Valid options: add, view, update, delete, exit")  # <Issue>: previously just said "Unknown command."

if __name__ == '__main__':
    main()

# code generated by the model
```